In [5]:
import pickle
import time
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import connectivity_measures as cm

In [6]:
### Contructing filters
### Butterworth filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

fs      = 1000.0

filter_parameters = {'theta':    {'N': 4, 'lowcut': 3.5,  'highcut': 7.5},
                    'alpha':     {'N': 4, 'lowcut': 8.0,  'highcut': 13.0},
                    'beta':      {'N': 4, 'lowcut': 13.0, 'highcut': 25.0},
                    'low_gamma': {'N': 6, 'lowcut': 25.0, 'highcut': 60}}

butterworths = dict()
for band in filter_parameters.keys():
    lowcut = filter_parameters[band]['lowcut']
    highcut = filter_parameters[band]['highcut']
    N = filter_parameters[band]['N']
    butterworths[band] = butter_bandpass(lowcut, highcut, fs, order=N)


In [7]:
IDs = ['SERT1597']

In [11]:
condition = 'epochs'

iCohs = {}
for mouse in IDs: 
    npys_dir = '/home/maspe/filer/SERT/' + mouse + '/npys/'
    print('\nLoading mouse {}...'.format(mouse))
    
    ### Loading data
    data = pickle.load(open(npys_dir + mouse + '.epochs', 'rb'))
    
    ### Loop
    filtered_bands = {'theta': {}, 'low_gamma': {}}
    iterator = 0
    for structure in data.keys(): 
        if iterator == 0:
            all_structures = data[structure][condition]
        else:
            all_structures = np.vstack((all_structures, data[structure][condition]))
        
        iterator += 1
        
    print('Filtering...')    
    for band in filtered_bands.keys():
        filtered = signal.filtfilt(b=butterworths[band][0], a=butterworths[band][1],
                                   x=all_structures, axis=1)
        
        print('Calculating iCoh for {} band...'.format(band))
        clock = time.time()

        n_epochs = filtered.shape[2]
        for epoch in range(n_epochs):
            if epoch == 0:
                icoh = cm.icoh(filtered[:,:,epoch], average = False)
            else:
                icoh = np.dstack((icoh, cm.icoh(filtered[:,:,epoch], average = False)))
        
        print('iCoh calculated in {} min'.format((time.time() - clock)) / 60)
        
        iCohs[band] = icoh

    
    pickle.dump(iCohs, open(npys_dir + mouse + '.icoh', 'wb'), protocol=2)

    
print('Done!')      


Loading mouse SERT1597...
Filtering...
Calculating iCoh...
iCoh calculated in 245.968783855 s
Calculating iCoh...
iCoh calculated in 242.087009907 s
Done!
